NOTEBOOK OVERVIEW

EVALUATING SUMMARIES OF SIMPLIFIED TEXT: In this notebook, we load in the test data and then take the first 5 rows to use for a small simplification experiment.

Using the finetuned T5 model (and tuned hyperparameters), we generate summaries for both the original and the legalese-decoded ('simplified') texts. Finally, for each of the 5 rows, we get a Rouge score comparing the summary of the original text with the target AND a Rouge score comparing the summary of the simplified text with the target. We export this as "legalese_df."

In [1]:
# mount to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# first, get the test data
import pandas as pd

test_file = "drive/MyDrive/266project/billsum_test.csv"
test_data = pd.read_csv(test_file)

In [4]:
# legalese "PREPROCESSING" done manually. load file here
pd.set_option('display.max_colwidth', None)
test_data = test_data.drop('Unnamed: 0', axis = 1)
test_data_5 = test_data.head(5)
legalese_file = "drive/MyDrive/266project/legalese.csv"
legalese_data = pd.read_csv(legalese_file)


In [5]:
legalese_df = test_data_5.join(legalese_data)
legalese_df
legalese_df.to_csv("drive/MyDrive/266project/legalese_df.csv")

In [7]:
!pip install -q rouge_score
!pip install -q evaluate
import numpy as np
import evaluate
import rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.7 MB/s eta 0:00:00


In [8]:
!pip install transformers
from transformers import AutoTokenizer  , TFAutoModel
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from pprint import pprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00


In [9]:
t5_model_name = 't5-base'
orig_t5_model = TFT5ForConditionalGeneration.from_pretrained(t5_model_name)
tokenizer = AutoTokenizer.from_pretrained(t5_model_name)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [10]:
# new HPs
def generate_candidates_HPs(orig, model):
  """test data is a pandas df"""
  input = tokenizer(orig, return_tensors = 'tf',padding = True).input_ids
  output = model.generate(input,
                          max_new_tokens = 150,
                          min_length=50,
                          num_beams = 1,
                          no_repeat_ngram_size = 2
                          )
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_t5_training_wrapper_model(my_t5, max_length):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_mask = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_mask')
    decoder_input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='labels')

    t5_logits = my_t5(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids)[0]

    model = tf.keras.models.Model(inputs=[input_ids, attention_mask, decoder_input_ids],
                                  outputs=[t5_logits])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

    return model

In [13]:
max_length = 168
checkpoint_dir = 'drive/MyDrive/266project/model_checkpoints/'
latest_model = 't5_billsum_weights.03.hdf5'
my_model = orig_t5_model
model_wrapper = build_t5_training_wrapper_model(my_model, max_length)
model_wrapper.load_weights(checkpoint_dir + latest_model)

In [16]:
# run my model on regular top 5
# run my model on legalese top 5
# then get rouge scores and show df
legalese_df


orig  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [17]:
#
%%timeit
legalese_df['original_candidates'] = legalese_df.apply(lambda x: generate_candidates_HPs(x.orig, my_model), axis = 1)


Token indices sequence length is longer than the specified maximum sequence length for this model (1812 > 512). Running this sequence through the model will result in indexing errors


3min 14s ± 5.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
rouge = evaluate.load('rouge')

orig_data_rouge = rouge.compute(predictions = legalese_df['original_candidates'],
                       references= legalese_df['target'])
orig_data_rouge

{'rouge1': 0.27170636287857736,
 'rouge2': 0.06403355420837445,
 'rougeL': 0.17230300105830917,
 'rougeLsum': 0.187117815873124}

In [19]:
%%timeit
legalese_df['candidates_simplified_input'] = legalese_df.apply(lambda x: generate_candidates_HPs(x.simplified, my_model), axis = 1)

2min 33s ± 10 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
simplified_data_rouge = rouge.compute(predictions = legalese_df['candidates_simplified_input'],
                       references= legalese_df['target'])
simplified_data_rouge

{'rouge1': 0.25778406628082207,
 'rouge2': 0.0731359601745564,
 'rougeL': 0.1736700622574435,
 'rougeLsum': 0.1934231486771966}

In [21]:
legalese_df

orig  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [22]:
legalese_df.to_csv("drive/MyDrive/266project/legalese_df_with_results.csv")

In [23]:
# metrics experiment take 2

def rougify(candidate,target):
  return rouge.compute(predictions=[candidate], references=[target])

legalese_df['rouge_simplified_candidates'] = legalese_df.apply(lambda x: rougify(x['candidates_simplified_input'],x['target']), axis = 1)


In [24]:
# sentence bert for each line
def bertify(target,candidate, model):
  cosine_scores = []
  target_embedding = model.encode(target)
  candidate_embedding = model.encode(candidate)
  cosine_score = util.cos_sim(target_embedding, candidate_embedding)
  cosine_scores.append(cosine_score)
  return (cosine_scores)

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util
bert_model = SentenceTransformer('all-MiniLM-L6-v2')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 800.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=cff925ae7ef6dd92d48361716d88bbc7b4c503b399ffff922c8c69e935d33ed6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [26]:
legalese_df['cosine_simplified_candidates'] = legalese_df.apply(lambda x: bertify(x['candidates_simplified_input'],x['target'],bert_model), axis = 1)


In [29]:
legalese_df['rouge_regular_candidates'] = legalese_df.apply(lambda x: rougify(x['original_candidates'],x['target']), axis = 1)


In [30]:
legalese_df['cosine_regular_candidates'] = legalese_df.apply(lambda x: bertify(x['original_candidates'],x['target'],bert_model), axis = 1)


In [31]:
legalese_df.to_csv("drive/MyDrive/266project/legalese_df_with_results_rouge_and_bert2.csv")

In [32]:
legalese_df

orig  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       